In [1]:
import pulp

In [2]:
digits = [d for d in range(10)]
values = [k for k in range(10)]
summands = [1,2]

In [3]:
# create problem
prob = pulp.LpProblem("lines", pulp.LpMaximize)

# summand 1
m = pulp.LpVariable("m",cat=pulp.LpInteger)
# summand 2
n = pulp.LpVariable("n",cat=pulp.LpInteger)
# x[i][j][k]=1 <=> digit i=k for summand j
x = pulp.LpVariable.dicts("x",(digits,summands,values),cat=pulp.LpBinary)
# z[i]=1 <=> digit i=0 for m and n
z = pulp.LpVariable.dicts("z",digits,cat=pulp.LpBinary)

# objective
prob+= m+n

# m and n in base 10
prob+= m == pulp.lpSum(k*x[i][1][k]*10**i for i in digits for k in values)
prob+= n == pulp.lpSum(k*x[i][2][k]*10**i for i in digits for k in values)

# one value per digit
for j in summands:
    for i in digits:
        prob+= pulp.lpSum(x[i][j][k] for k in values) == 1

# all non 0 digits are distinct
for k in values:
    if k>0:
        prob+= pulp.lpSum(x[i][j][k] for i in digits for j in summands) <= 1

# digitwise sum must produce carry
for i in digits:
    if i==0:
        prob+= pulp.lpSum(k*x[i][j][k] for j in summands for k in values) >= 10
    else:
        prob+= pulp.lpSum(k*x[i][j][k] for j in summands for k in values) >= 9*(1-z[i])

# z[i] = 1 => digit i=0
for i in digits:
    for j in summands:
        prob+= z[i] <= x[i][j][0]

# if there is a double 0, the digits on the left are also 0
for i in digits:
    if i<max(digits):
        prob+= z[i] <= z[i+1]

In [4]:
prob.solve()

1

In [5]:
print("m + n = ",int(pulp.value(m)),"+",int(pulp.value(n)),"=",int(pulp.value(prob.objective)))

m + n =  9348 + 7652 = 17000
